In [46]:
from altaipony.lcio import from_path
import pandas as pd
import numpy as np

In [5]:
flc = from_mast(142052876, c=8, mission="TESS", mode="LC")

11% (1987/17755) of the cadences will be ignored due to the quality mask (quality_bitmask=175).
11% (1987/17755) of the cadences will be ignored due to the quality mask (quality_bitmask=175).
/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:59: ResourceWarning: unclosed file <_io.FileIO name='/home/ekaterina/.lightkurve-cache/mastDownload/TESS/tess2019032160000-s0008-0000000142052876-0136-s/tess2019032160000-s0008-0000000142052876-0136-s_lc.fits' mode='rb' closefd=True>
  return _from_mast_TESS(targetid, c, **kwargs)


In [6]:
flcd = flc.detrend("savgol").to_fits("pony_142052876_08.fits")

In [37]:
ID = 142052876
c = 8
flc = from_path("pony_{}_{:02d}.fits".format(ID, c), mode="AltaiPony", mission="TESS")

/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='pony_142052876_08.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)


In [47]:
flc = flc[np.where(flc.time!=0.)]

In [48]:
df = pd.read_csv("ucd_flares.csv")

In [49]:
dff = df[(df.EPIC == ID) & (df.Campaign == c)]

In [50]:
LC = pd.DataFrame({"time":flc.time, "flux":flc.detrended_flux})
LC["flares"] = 0
for i, row in dff.iterrows():
    LC.flares[int(row.istart): int(row.istop)] = 1.
    

/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [111]:


from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Band, BoxAnnotation
output_notebook()




Loading BokehJS ...

In [112]:
source = ColumnDataSource(LC)

In [133]:
p = figure(plot_height=350, title="Pie Chart", toolbar_location="right",
           tools= "pan,wheel_zoom,box_zoom,reset")

for i, row in dff.iterrows():
    LC.index = LC.time
    LCsub = LC.iloc[int(row.tstart)-1: int(row.tstop)+1,:]
    LCsub["upper"] = np.max(LCsub.time)
    LCsub["lower"] = np.min(LCsub.time)
    subset = ColumnDataSource(LCsub)
   # p.quad(top=np.max(LCsub.flux), bottom=np.min(LCsub.flux),
   #           left=np.min(LCsub.time), right=np.max(LCsub.time), color="#BBBBBB", alpha=0.5)
#    p.line(x='time', y='flux', source=subset,line_color="black")
    #band = Band(base='time', lower='lower', upper='upper', source=subset, level='underlay',
    #        fill_alpha=1.0, line_width=1, line_color='red', dimension="width")
    #p.add_layout(band)
    p2 = BoxAnnotation( left=np.min(LCsub.time), right=np.max(LCsub.time), fill_alpha=0.5, fill_color='red')

    p.renderers.extend([p2])
    
p.line('time', 'flux', source=source,line_color="black")
p.xgrid.visible = False
p.ygrid.visible = False

/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ekaterina/Documents/001_Science/Flares_in_Clusters_II/flaresinclustersii/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ValueError: cannot insert time, already exists

In [132]:
show(p)